<a href="https://colab.research.google.com/github/pushyag1/BigData_PySpark/blob/main/Pyspark_All_codes_Compiled_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title PySpark Setup(run me!)
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

import findspark
findspark.init()

findspark.find()

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark



Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [2]:
#@title Dynamic Input(select your choices and run the cell)
Date_to_calculate_age = '2020-01-01' #@param {type:"date"}
Choose_a_model_version = 'V24' #@param ["V22","V24", "RX", "ESRD-P1","ESRD-P2"]
Choose_a_model_year = '2020' #@param ["2020","2021"]
Sex_Age_edits_required = 'Yes' #@param ["Yes","No"]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title Age Calculation
import pandas 
import os
import pyspark.sql.functions as func
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import struct
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf


#Loading Persons file
df1 = pandas.read_csv ("/content/Person-file2.csv")
#df1=df.drop(['Unnamed: 8', 'Unnamed: 9','Unnamed: 10'], axis=1)
df1 = spark.createDataFrame(df1)

df2=df1.withColumn("Age",(datediff(to_date(lit(Date_to_calculate_age)),("DOB")))/366)
df3 = df2.withColumn("Age", func.round(df2["Age"], 2).cast('integer'))

#Condition for AGE and OREC
def func(Age, OREC):
  if Age == 64 and OREC == 0:
    return 65
  elif Age < 0 :
    return 0
  return Age


func_udf = udf(func, IntegerType())
df4 = df3.withColumn('new_column',func_udf(df3['Age'], df3['OREC']))

drop_list = [ 'Age',]
sdf5=df4.select([column for column in df4.columns if column not in drop_list])

df_final = sdf5.withColumnRenamed("new_column", "Age")

df_final.show()

+---------+----------+------+----+--------+-------+----+-----+---+
|Person ID|       DOB|Gender|OREC|LTIMCAID|NEMCAID|ESRD|MCAID|Age|
+---------+----------+------+----+--------+-------+----+-----+---+
|      111|1940-03-18|     2|   1|       0|      0|   1|    0| 79|
|      112|1947-02-23|     1|   2|       0|      0|   0|    0| 72|
|      113|1950-04-06|     1|   1|       0|      1|   1|    0| 69|
|      114|1948-09-05|     2|   0|       1|      1|   1|    0| 71|
|      115|1940-04-30|     2|   3|       0|      0|   0|    0| 79|
|      116|1951-03-25|     1|   0|       1|      1|   1|    1| 68|
|      117|1947-02-03|     1|   0|       0|      0|   0|    1| 72|
|      118|1949-04-11|     1|   2|       0|      0|   0|    0| 70|
|      119|1954-09-12|     1|   1|       0|      0|   1|    0| 65|
|      120|1941-01-16|     1|   1|       1|      1|   0|    0| 78|
|      121|1943-09-29|     1|   3|       0|      0|   0|    0| 76|
|      122|1945-03-22|     2|   0|       0|      0|   0|    0|

In [4]:
#@title Demography Variable

import pandas as pd
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import col, expr, when
import pyspark.sql.functions as F
from pyspark.sql.types import *

data = df_final

demo = (spark.read.format("csv").options(header="true").load("/content/Demography Variable Calculations.csv"))

demo = demo.filter((demo['Version'] == Choose_a_model_version) & (demo['Year'] == Choose_a_model_year))
data = data.withColumn("Year", lit(str(Choose_a_model_year)))
data = data.withColumn("Version", lit(str(Choose_a_model_version)))

for i in demo.collect():
  data = data.withColumn(i[2], expr(i[3]))

data.show()

#data=data.toPandas()

+---------+----------+------+----+--------+-------+----+-----+---+----+-------+------+-----+------+------+------+------+------+------+------+------+------+------+------+----------+-----+------+------+------+------+------+------+------+------+------+------+------+----------------+----------------+---------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+----------------+----------------+------+
|Person ID|       DOB|Gender|OREC|LTIMCAID|NEMCAID|ESRD|MCAID|Age|Year|Version|DISABL|F0_34|F35_44|F45_54|F55_59|F60_64|F65_69|F70_74|F75_79|F80_84|F85_89|F90_94|F95_GT|LTIMCAID_O|M0_34|M35_44|M45_54|M55_59|M60_64|M65_69|M70_74|M75_79|M80_84|M85_89|M90_94|M95_GT|MCAID_NORIGDIS  |MCAID_ORIGDIS   |NE_ORIGDS|NEF0_34|NEF35_44|NEF45_54|NEF55_59|NEF60_64|NEF65|NEF66|NEF67|NEF68|NEF69|NEF70_74|NEF75

In [5]:
data = data.withColumnRenamed("Version","V")

In [ ]:
data.show()

+---------+----------+------+----+--------+-------+----+-----+---+----+---+------+-----+------+------+------+------+------+------+------+------+------+------+------+----------+-----+------+------+------+------+------+------+------+------+------+------+------+----------------+----------------+---------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+----------------+----------------+------+
|Person ID|       DOB|Gender|OREC|LTIMCAID|NEMCAID|ESRD|MCAID|Age|Year|  V|DISABL|F0_34|F35_44|F45_54|F55_59|F60_64|F65_69|F70_74|F75_79|F80_84|F85_89|F90_94|F95_GT|LTIMCAID_O|M0_34|M35_44|M45_54|M55_59|M60_64|M65_69|M70_74|M75_79|M80_84|M85_89|M90_94|M95_GT|MCAID_NORIGDIS  |MCAID_ORIGDIS   |NE_ORIGDS|NEF0_34|NEF35_44|NEF45_54|NEF55_59|NEF60_64|NEF65|NEF66|NEF67|NEF68|NEF69|NEF70_74|NEF75_79|NEF8

In [6]:
#@title CC_Mapping

import pandas as pd
import numpy as np
import time
from pyspark.sql.functions import length,col,trim
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col

df_diagnosis = pd.read_csv('/content/Person_DiagonsisCodes2.csv')
df_diagnosis = spark.createDataFrame(df_diagnosis)
df_diagnosis = df_diagnosis.withColumn('DIAGNOSIS CODE',trim(col("DIAGNOSIS CODE")))

df_CC_mapping = pd.read_csv('/content/CC Mapping.csv')
label_schema = StructType([
    StructField("YEAR", StringType()),
    StructField("VERSION", StringType()),
    StructField("DIAGNOSIS CODE", StringType()),
    StructField("CC", IntegerType())
])
df_CC_mapping = spark.createDataFrame(df_CC_mapping,schema= label_schema)


start=time.time()
# for year in model_years:
#   for version in versions:
cond1= col('YEAR')== str(Choose_a_model_year)
cond2= col('VERSION')== str(Choose_a_model_version)
df_temp= df_CC_mapping.where(cond1 & cond2)

# df_temp.show(5)
df_temp= df_temp.toDF('YEAR','VERSION','D_DIAG CODE','CC')

h= df_temp.join(df_diagnosis,(df_diagnosis["DIAGNOSIS CODE"] == df_temp["D_DIAG CODE"]),how='right')  # Mtachin diagnosis codes from diagnosis input file and CC_mapping file

unique_HCC= sorted([i.CC for i in df_temp.select('CC').distinct().collect()]) #Finding all unique values of HCC and storing it in a list to generate HCC columns later

CC_list = h.select("CC").rdd.flatMap(lambda x: x).collect()

p_id = h.select("PERSON ID").rdd.flatMap(lambda x: x).collect()

diag_code= h.select("DIAGNOSIS CODE").rdd.flatMap(lambda x: x).collect()


df_main= pd.DataFrame()
df_main['PERSON ID']= p_id
df_main['DIAGNOSIS CODE']= diag_code 

df_main['HCC']= CC_list  # CC values obtained after mapping from Part 2.
df_main['HCC'] = list(df_main['HCC'].astype(np.float).astype("Int32"))

df_main['MODEL YEAR']= str(Choose_a_model_year)
df_main['Version']=str(Choose_a_model_version)

# df_main
if 'RX' in Choose_a_model_version:
  columns_df=[]
  for value in (unique_HCC):
    columns_df.append('RXHCC'+str(value)) # Example: RXHCC1,RXHCC2,RXHCC3 etc
  #print(columns_df)
  for colum in range(len(columns_df)):
    df_main[columns_df[colum]]=0  #adding zero's to every column 

  for v in range(len(df_main)):                         
    df_main.at[v,'RXHCC'+ str(df_main.iloc[v][2])]=1   # Adding 1's to columns. Example if df_main['HCC'][10]= 22, then adding a 1 in HCC10 column in the 9th row. 

  df_main = df_main.drop(df_main.columns[-1],axis=1)
  #df_main.to_excel(str(Choose_a_model_version)+'_'+str(Choose_a_model_year)+'.xlsx')


else:
  # Part 5: Creating new columns with column names as HCC values obtained from Part 4
  columns_df=[]
  for value in (unique_HCC):
    columns_df.append('HCC'+str(value)) # Example: HCC1,HCC2,HCC3 etc
  #print(columns_df)
  for colum in range(len(columns_df)):
    df_main[columns_df[colum]]=0  #adding zero's to every column 

  for v in range(len(df_main)):                         
    df_main.at[v,'HCC'+ str(df_main.iloc[v][2])]=1   # Adding 1's to columns. Example if df_main['HCC'][10]= 22, then adding a 1 in HCC10 column in the 9th row. 

  df_main = df_main.drop(df_main.columns[-1],axis=1)
  #df_main.to_excel(str(Choose_a_model_version)+'_'+str(Choose_a_model_year)+'.xlsx')

print(time.time()-start)
df_main= df_main.fillna(0)
spark_df_main= spark.createDataFrame(df_main)
spark_df_main.printSchema()

235.5824248790741
root
 |-- PERSON ID: long (nullable = true)
 |-- DIAGNOSIS CODE: string (nullable = true)
 |-- HCC: long (nullable = true)
 |-- MODEL YEAR: string (nullable = true)
 |-- Version: string (nullable = true)
 |-- HCC1: long (nullable = true)
 |-- HCC2: long (nullable = true)
 |-- HCC6: long (nullable = true)
 |-- HCC8: long (nullable = true)
 |-- HCC9: long (nullable = true)
 |-- HCC10: long (nullable = true)
 |-- HCC11: long (nullable = true)
 |-- HCC12: long (nullable = true)
 |-- HCC17: long (nullable = true)
 |-- HCC18: long (nullable = true)
 |-- HCC19: long (nullable = true)
 |-- HCC21: long (nullable = true)
 |-- HCC22: long (nullable = true)
 |-- HCC23: long (nullable = true)
 |-- HCC27: long (nullable = true)
 |-- HCC28: long (nullable = true)
 |-- HCC29: long (nullable = true)
 |-- HCC33: long (nullable = true)
 |-- HCC34: long (nullable = true)
 |-- HCC35: long (nullable = true)
 |-- HCC39: long (nullable = true)
 |-- HCC40: long (nullable = true)
 |-- HCC46: l

In [ ]:
#spark_df_main.show()

## @CC-Override

In [ ]:
#@title CC-Override
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = data.withColumnRenamed("Person ID", "PERSON ID")


In [8]:
df = data.join(spark_df_main, 'PERSON ID', 'inner')
# list3.show()

In [9]:
columns_to_drop = ['Year','V']
df = df.drop(*columns_to_drop)

In [10]:
df = df.withColumnRenamed("DIAGNOSIS CODE","DIAG")

In [11]:
df = df.withColumnRenamed("Gender","SEX")

In [12]:
df = df.withColumnRenamed("Age","AGEF")

In [ ]:
#df.show()

In [14]:

data = df
    
overide_df = (spark.read.format("csv").options(header="true").load("/content/CC Override Rules.csv"))
overide_df = overide_df.withColumnRenamed("MODEL YEAR","MODEL_YEAR")
overide_df = overide_df.filter((overide_df.VERSION.isin(Choose_a_model_version))&(overide_df.MODEL_YEAR.isin(Choose_a_model_year)))
for i in overide_df.collect():
  data = data.withColumn('HCC', expr(i[3]))
#data.toPandas().to_excel('Merged_CCOveride.xlsx')

In [15]:
df_CCmerged= data

In [17]:
from pyspark.sql.functions import col

In [20]:
result = df_CCmerged.where(col("DIAG") == "F3481")

In [21]:
result.show()

+---------+----------+---+----+--------+-------+----+-----+----+------+-----+------+------+------+------+------+------+------+------+------+------+------+----------+-----+------+------+------+------+------+------+------+------+------+------+------+----------------+----------------+---------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+----------------+----------------+------+-----+---+----------+-------+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+

-- END OF CC-OVERRIDE with single row for each person ID for for each year and version

HCC HIERARCHY STARTS HERE

In [ ]:
data = pd.read_excel('/content/HCC Hierarchy.xlsx')   #Loading Files
data1 = spark.createDataFrame(data)                                       #Making Spark Dataframe
data1.show()                                                              #Making Spark Dataframe
merged_cc = df_CCmerged #Loading File Created by HCC Hierarchy Team

versions = data['MODEL_VRSN'].unique()                                    #Taking unique values to iterate later
years = data['MODEL_YR'].unique()                                         #Taking unique values to iterate later

from pyspark.sql import functions as F


#Section to create hierarchy table for all unique years and version being passed from above. This ensure it to be dynamic.
lis = {}
for k in range(len(years)):
  for l in range(len(versions)):
    var = str(years[k]) + '_' +  str(versions[l])
    lis[var] = []
    data_2020_test = data[(data['MODEL_YR'] == years[k]) & (data['MODEL_VRSN'] == versions[l])]
    merged_cc_2020_test = merged_cc[(merged_cc['MODEL YEAR'] == years[k]) & (merged_cc['Version'] == versions[l]) ]
    merged_cc_2020_spark_test = spark.createDataFrame(merged_cc_2020_test)
    for i in range(len(data_2020_test)):
      merged_cc_2020_spark1 = 0
      if list(data_2020_test['HIGHER_HCC'])[i] in merged_cc_2020_spark_test.columns:
        merged_cc_2020_spark1 = merged_cc_2020_spark_test.withColumn(list(data_2020_test['LOWER_HCC'])[i], F.when((F.col(list(data_2020_test['HIGHER_HCC'])[i])==1) & (F.col(list(data_2020_test['LOWER_HCC'])[i])== 1) & (F.col('VERSION')== list(data_2020_test['MODEL_VRSN'])[i]) ,0).otherwise(F.col(list(data_2020_test['LOWER_HCC'])[i])))
        # print(i)
    lis[var].append(merged_cc_2020_spark1)



# Creating seperate files for different years and versions and saving excel files.
mer_frames = []
for i in lis.keys():
  print(i)
  if lis[i][0]!=0:  
    frame = lis[i][0]
    frame = frame.toPandas()
    mer_frames.append(frame)
    frame.to_excel('/content/gdrive/My Drive/' + str(i) + '.xlsx')
result = pd.concat(mer_frames)

#Savin
result.to_excel('/content/gdrive/My Drive/Merged_file_for_Akarshika_updated.xlsx')

END OF HCC HIERARCHY



Interaction Variable Calculation



In [ ]:
 #loading libraries
import pandas as pd
import os
import pyspark.sql.functions as func

In [ ]:
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import col, expr, when
import pyspark.sql.functions as F
from pyspark.sql.types import *

result= pd.read_csv('/content/Merged_file_for_HCC_updated.csv')
df1_copy = result.copy()
data_interaction = spark.createDataFrame(df1_copy)

interaction_demo = (spark.read.format("csv").options(header="true").load("/content/Interaction Variable Calculations - 1.csv"))

interaction_demo = interaction_demo.filter((interaction_demo['MODEL VRSN'] == Choose_a_model_version) & (interaction_demo['MODEL YR'] == Choose_a_model_year))
data_interaction = data_interaction.where(F.col('MODEL YEAR') == Choose_a_model_year)
data_interaction = data_interaction.withColumn("Version", lit(str(Choose_a_model_version)))

for i in interaction_demo.collect():
  if i[2] in result.columns:
      data_interaction = data_interaction.withColumn(i[2], expr(i[3]))

data_interaction.show()
data_interaction.toPandas().to_excel('Interaction_Variable_Calculation.xlsx') 


+----------+------------+---------+-----+---+----------+-------+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+---+----+--------+-------+----+-----+----+------------+-----------+------------+-----------+--------+--------+------+---------------+-----------------+--------------+----------------+----------------------+--------------------+-------------------------+-----------------------+-----------------+---------------+--------------------+------------------+------------

End of Interaction Variable Calculation

Score Calculation


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
filePath = 'Interaction_Variable_Calculation.xlsx'

In [ ]:
dfFinal = pd.read_excel(filePath)
del dfFinal['DOB']
del dfFinal['SEX']
del dfFinal['OREC']
del dfFinal['LTIMCAID']
del dfFinal['NEMCAID']
del dfFinal['ESRD']
del dfFinal['MCAID']
del dfFinal['AGEF']

In [ ]:
dfColumnNames = dfFinal.columns.values.tolist()
print(dfColumnNames)

In [ ]:
# Fetch score sheet
scoreSheetPath = 'Score Variables.xlsx'   #Path to access the score variables from drive
scoreSheetDF = pd.read_excel(scoreSheetPath)

In [ ]:
# Data from Score sheet for further reference
modelYearScoreList = scoreSheetDF['Model Year'].tolist()                
modelVersionScoreList = scoreSheetDF['Model Version'].tolist()
variableScoreList = scoreSheetDF['Variable'].tolist()
coeffValueScoreList = scoreSheetDF['Coefficient Value'].tolist()
scoreTypeList = scoreSheetDF['Score Type'].tolist()


# Data from the sheet generated by executing above cells
personIdList = dfFinal['PERSON ID'].tolist()
modelYearsFinalDF = dfFinal['MODEL YEAR'].tolist()
modelVersionFinalDF = dfFinal['Version'].tolist()
print(modelYearsFinalDF)

In [ ]:
#Note: execute only once
N = 7
removedColumns = dfFinal[:N]
del dfColumnNames[0:N]  #Removing other columns and keeping Variables related columns for easy operation
print(dfColumnNames)

In [ ]:
#Code for picking person ID's
modelYearnpListDF = np.array(modelYearsFinalDF)
modelVersionnpListDF = np.array(modelVersionFinalDF)
yearMatchIndexesDF = np.where(modelYearnpListDF == int(Choose_a_model_year))[0]         # Fetching all matched indexes of selected Year
versionMatchIndexesDF = np.where(modelVersionnpListDF == Choose_a_model_version)[0]   # Fetching all matched indexes of selected version
#print(matchedIndexesOfPersonID)

match2 = set(yearMatchIndexesDF) & set(versionMatchIndexesDF)                      # Taking out indexes which have both the selected year and selected versions
matchedIndexesOfPersonID = list(match2)

In [ ]:
#Code for picking All matched Score types from SCore list which have both the selected year and selected versions
modelYearsList = np.array(modelYearScoreList)
modelVersionsList = np.array(modelVersionScoreList)
yearMatchIndexes = np.where(modelYearsList == int(Choose_a_model_year))[0]
versionMatchIndexes = np.where(modelVersionsList == Choose_a_model_version)[0]

match = set(yearMatchIndexes) & set(versionMatchIndexes)
matchedIndexes = list(match)

allMatchedScoreTypes = []
for i in range(len(matchedIndexes)):
  allMatchedScoreTypes.append(scoreTypeList[matchedIndexes[i]+2])

allMatchedScoreTypesSet = set(allMatchedScoreTypes)
allMatchedList = list(allMatchedScoreTypesSet)
print(Choose_a_model_year)

In [ ]:
import numpy as np

for i in range(len(dfColumnNames)):   # Go through al columns
  varValList = dfFinal[dfColumnNames[i]].tolist()  # Fetch all columns data into list
  for j in  range(len(varValList)):    # Go through each column
    if varValList[j] != 1 and varValList[j] != 0:               # if Nan replace it with 0
          dfFinal.loc[j, dfColumnNames[i]] = 0

print(dfFinal)

In [ ]:
# Create a new Data Frame
values = np.array(variableScoreList)
finalAmplifiedData = []
scoreData = []
for i in range(len(matchedIndexesOfPersonID)):
  for scoreType in range(len(allMatchedList)):
    eachRow = []
    eachRow.append(personIdList[matchedIndexesOfPersonID[i]])
    eachRow.append(Choose_a_model_year)
    eachRow.append(Choose_a_model_version)
    eachRow.append(allMatchedList[scoreType])
    for j in range(len(dfColumnNames)):
      varValList = dfFinal[dfColumnNames[j]].tolist()  # Fetch all columns data into list
      searchval = dfColumnNames[j]       # get for the variable name
      ii = np.where(values == searchval)[0] # Search for it's existence and get the matching indexes
      for k in range(len(ii)):
       # print(ii[k]+2) ii[k]+2
        if modelYearScoreList[ii[k]] == int(Choose_a_model_year) and modelVersionScoreList[ii[k]] == Choose_a_model_version and scoreTypeList[ii[k]] == allMatchedList[scoreType]:
          varValList[matchedIndexesOfPersonID[i]] =  1 * coeffValueScoreList[ii[k]]
      if varValList[matchedIndexesOfPersonID[i]] == 1:
          varValList[matchedIndexesOfPersonID[i]] = 0
      eachRow.append(varValList[matchedIndexesOfPersonID[i]])
    finalAmplifiedData.append(eachRow)

In [ ]:
dfColumnNames.insert(0, 'PERSON ID')
dfColumnNames.insert(1, 'Model Year') 
dfColumnNames.insert(2, 'Model Version') 
dfColumnNames.insert(3, 'Score Type') 

In [ ]:
df123 = pd.DataFrame(finalAmplifiedData, columns=dfColumnNames)
print(df123)

In [ ]:
finalPersonIdList = df123['PERSON ID'].tolist()
finalModelYear = df123['Model Year'].tolist()
finalModelVersion = df123['Model Version'].tolist()
FinalScoreType = df123['Score Type'].tolist()

In [ ]:
del df123['PERSON ID']
del df123['Model Year']
del df123['Model Version']
del df123['Score Type']

In [ ]:
df123["Total Score"] = df123.sum(axis=1)


In [ ]:
finalTotalScore = df123['Total Score'].tolist()

In [ ]:
finalDF = pd.DataFrame(list(zip(finalPersonIdList, finalModelYear, finalModelVersion, FinalScoreType, finalTotalScore)),
               columns =['PERSON ID', 'Model Year', 'Model Version', 'Score Type', 'Total Score'])


In [ ]:
data = spark.createDataFrame(finalDF)
data.toPandas().to_excel('final_Sheet.xlsx') 